In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2

#import glob2

from numba import jit
import tables
import scipy
import csv

#import deeplabcut


Autosaving every 180 seconds


In [2]:
video_name = "/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/2020-3-9_08_18_49_128168_compressed_snippet.avi"

data= np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/traces_means.npz')

tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']
print ("DONE LOADING")

DONE LOADING


In [10]:
# BREAKUP LARGE JUMPS
tracesx = data['tracesx_mean']
tracesy = data['tracesy_mean']

def delete_jumps(tracesx, tracesy):
    #tracesx_ip=tracesx.copy()
    #tracesy_ip=tracesy.copy()
    for k in range(tracesx.shape[1]):

        # for big jumps, set intermediate values to nans
        max_jump = 50
        for p in range(0,tracesx.shape[0]-1,1):
            distx = tracesx[p,k]-tracesx[p+1,k]
            disty = tracesy[p,k]-tracesy[p+1,k]
            dist = np.sqrt(distx**2+disty**2)
            #print ("Dist; ", dist)
            if dist>max_jump:
                #print (snippets[p,0],snippets[p,1])
                tracesx[p:p+2,k]= np.nan
                tracesy[p:p+2,k]= np.nan
                #print ("found large jump: ", p/25.)
            #else:
            #    print (dist, "not merging")


       # break
        #np.isnan(x_loc) or np.isnan(y_loc):
    print ("DONE FIXING JUMPS")
    #print ("tracex: ", tracesx[:100,0])
    
    return tracesx, tracesy

tracesx, tracesy = delete_jumps(tracesx,tracesy)
#print ("tracex_ip: ", tracesx_ip[:100,0])

#tracesx = tracesx_ip.copy()
#tracesy = tracesy_ip.copy()


DONE FIXING JUMPS


In [16]:
# INTERPOLATE TRACES
# tracesx = data['tracesx_mean']
# tracesy = data['tracesy_mean']

# TODO Add time threshold also; 
def interpolate_traces(tracesx, tracesy, dist_threshold=10., time_threshold=10):
    print (tracesx.shape)
    tracesx_ip=tracesx.copy()
    tracesy_ip=tracesy.copy()
    for k in range(tracesx_ip.shape[1]):
        start = False
        end = False
        idx = np.where(~np.isnan(tracesx_ip[:,k]))[0]

        # make snippets
        snippets=[]
        locs = []
        start = idx[0]
        #for p in range(1,idx.shape[0],1):
        p=1
        ctr=0
        while True:
            if idx[p]-idx[p-1]>1:
                end = idx[p-1]
                snippets.append([start,end])
                start = idx[p]   

                # find locations of start and end
                locs.append([])
                locs[ctr].append([tracesx_ip[start,k], tracesy_ip[start,k]])
                locs[ctr].append([tracesx_ip[end,k], tracesy_ip[end,k]])

                # bump 2 over
                p+=2
                ctr+=1
            else:
                p+=1

            if p>=idx.shape[0]:
                break

        snippets = np.array(snippets)
        #print (snippets.shape)
        #print (snippets)
        locs = np.array(locs)
    #     print (locs.shape)
    #     print (locs[0])

        # fill in space between snippets if euclidean distance < threshold
        #threshold = 10.
        # AND IF NOT TOO FAR APART; 
        for p in range(snippets.shape[0]-1):
            # Skip if interpolate across too long a sgement
            if (snippets[p-1,1] - snippets[p,0])>time_threshold:
                continue
            dist = np.linalg.norm(locs[p,1]-locs[p+1,0])
            #print ("Dist; ", dist)
            if dist<dist_threshold:
                #print (snippets[p,0],snippets[p,1])
                tracesx_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesx_ip[snippets[p-1,1],k]
                tracesy_ip[snippets[p-1,1]:snippets[p,0]+1,k]= tracesy_ip[snippets[p-1,1],k]
            #else:
            #    print (dist, "not merging")

    return tracesx_ip, tracesy_ip

dist_threshold = 10.
time_threshold = 10
tracesx, tracesy = interpolate_traces(tracesx, tracesy, dist_threshold, time_threshold)

(16572, 4)


In [ ]:
# AUTO SWITCH LOCATIONS BASED ON PROXIMTIY



In [17]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
import matplotlib.gridspec as gridspec

global current_index, selected_animal, tracesx, tracesy

# reload data
#tracesx = tracesx_ip.copy() #data['tracesx_mean'].copy()
#tracesy = tracesy_ip.copy() #data['tracesy_mean'].copy()


# set plotting figure
fig = plt.figure()
gs = gridspec.GridSpec(4, 4)
gs.tight_layout(fig)

clrs = ['red','blue','cyan','green']
sizes = np.ones(4,'int32')*50
sizes[0] = 250
history = np.ones((4,5),'int32')+100

# save current frame of video
current_index = []
current_index.append(0)

# save selected animal radio button 
selected_animal = []
selected_animal.append(0)

# initialize start and ends of video to be analyzed
frame_rate = 25
original_vid = cv2.VideoCapture(video_name)

# initialize video
loc = 0
original_vid.set(cv2.CAP_PROP_POS_FRAMES, loc)
ret, frame = original_vid.read()
n_frames = int(original_vid.get(cv2.CAP_PROP_FRAME_COUNT))

# initialize scatter plots
ax_video = plt.subplot(gs[:, 1:4])
img = ax_video.imshow(frame)
ax_video.set_xlim(150,1200)
ax_video.set_ylim(50,800)

axes_animals=[]
for k in range(4):
    x_loc = tracesx[0,k]
    y_loc = tracesy[0,k]
    if np.isnan(x_loc) or np.isnan(y_loc):
        x_loc=100
        y_loc=100
    axes_animals.append(ax_video.scatter(x_loc, y_loc,c=clrs[k]))

# plot traces
ax_tracesx = plt.subplot(gs[:2, :1])
ax_tracesx.set_title("DLC-x")
init_length = 10*25
t = np.arange(init_length)/25.
ax1 = []
for k in range(4):
    temp, = ax_tracesx.plot(t,tracesx[:init_length,k],c=clrs[k])
    ax1.append(temp)

ax_tracesx.set_xticks([])
ax_tracesx.set_ylim(0,1280)

# plot y traces
ax_tracesy = plt.subplot(gs[2:, :1])
ax_tracesy.set_title("DLC-y")
ax2 = []
for k in range(4):
    temp, = ax_tracesy.plot(t,tracesy[:init_length,k],c=clrs[k])
    ax2.append(temp)

ax_tracesy.set_xlabel("Time (sec)")
ax_tracesy.set_ylim(0,1024)
# add 
#ax.margins(x=0)

##location of sliders
axcolor = 'lightgoldenrodyellow'
axfreq = plt.axes([0.01, 0.01, 0.94, 0.03], facecolor=axcolor)
#axamp = plt.axes([0.25, 0.15, 0.65, 0.03], facecolor=axcolor)

sfreq = Slider(axfreq, 'Frame', 0, n_frames, valinit=0, valstep=1, dragging=True)
#sfreq = Slider(axfreq, 'Frame',0.20, 18, valinit=0, dragging=True,  fc='blue')    
#sfreq.val=val


# update the slider
def update(val):
    #amp = samp.val
    index = int(sfreq.val)
    current_index[0] = index
    print ("current indx updated:", current_index)
    
    # grab frame and update it
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, index)
    ret, frame = original_vid.read()
    img.set_data(frame)

    # update scatter plots
    for k in range(4):
        x_loc = tracesx[index,k]
        y_loc = tracesy[index,k]
        
        # If no location inferred, freeze and turn off labels
        if np.isnan(x_loc) or np.isnan(y_loc):
            axes_animals[k].set_alpha(0.0)
            continue
        else:
            axes_animals[k].set_alpha(1.0)
            axes_animals[k].set_sizes([sizes[k]])
            axes_animals[k].set_offsets(np.c_[x_loc,y_loc])
    
    # update trace plots
    ax_tracesx.clear()
    ax_tracesx.set_ylim(0,1280)

    ax_tracesy.clear()
    ax_tracesy.set_ylim(0,1024)
    for k in range(4):
        ax_tracesx.plot(t+current_index[0]/25., tracesx[index:index+init_length,k],
                        linewidth=sizes[k]//50,
                       c=clrs[k])
        #ax1[k].set_linewidth(sizes[k]//50)
        #ax2[k].set_ydata(tracesy[index:index+init_length,k])    
        #ax2[k].set_linewidth(sizes[k]//50)
        ax_tracesy.plot(t+current_index[0]/25., tracesy[index:index+init_length,k], 
                        linewidth=sizes[k]//50,
                       c=clrs[k])
    fig.canvas.draw_idle()

sfreq.on_changed(update)
#samp.on_changed(update)

# resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
# button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

# def reset(event):
#     sfreq.reset()
#     #samp.reset()
# button.on_clicked(reset)

# ANIMAL SELECTOR
labels = ["female (red)", "male (blue)", "pup1 (cyan)", "pup2 (green)"]
rax1 = plt.axes([0.01, 0.8, 0.07, 0.1], facecolor=axcolor)
animal_buttons = RadioButtons(rax1, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax1.set_title('Select')
    
def animal_select(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    sizes[:] = 50
    for k in range(4):
        if labels[k] == label:
            break
    sizes[k]=250
    selected_animal[0]=k
    
    
animal_buttons.on_clicked(animal_select)

# ANIMAL SWITCHER
rax2 = plt.axes([0.01, 0.65, 0.07, 0.1], facecolor=axcolor)
animal_switch = RadioButtons(rax2, (labels[0],labels[1],labels[2],labels[3]), active=0)
rax2.set_title('Switch')

def animal_switch_function(label):
    #l.set_color(label)
    fig.canvas.draw_idle()
    for k in range(4):
        if labels[k] == label:
            break
    switch_id = k
    print ("switching to : ", labels[switch_id])
    print ("current index inside switch:", current_index)

    # search active trace snippet
    # search forward

    def find_snippet(current_index, tracesx, id_):
        end=n_frames
        for k in range(current_index[0]+1, n_frames,1):
            if np.isnan(tracesx[k,id_]):
                end = k
                break
        if k==(n_frames-1):
            end = n_frames-1

        # search backward
        start=0
        for k in range(current_index[0],0,-1):
            if np.isnan(tracesx[k,id_]):
            #if tracesx[k, current_index[0]]==9999:
                start = k
                break

        if k==(0):
            start = 0
    
        return end, start
    
    end2, start2 = find_snippet(current_index, tracesx, switch_id)
            
    print ("Switch animal id: ", switch_id, ", start/end: ", start2, end2)
    
    end1, start1 = find_snippet(current_index, tracesx, selected_animal[0])
    
    print ("selected animal id: ", selected_animal[0], ", start/end: ", start1, end1)
        
        
    # DATA SWITCH HAS TO BE SAFE FOR ALL DOWNSTREAM 
    # SO NEED TO CHECK THE ID THAT"S BEING SWAPPED TO MAKE SURE IT DOESN"T GET DELETED;
    # switch the data
    tempx = tracesx.copy()
    tempy = tracesy.copy()

    # replace switch id info
    tracesx[start1:end1, switch_id] = tracesx[start1:end1, selected_animal[0]]
    tracesy[start1:end1, switch_id] = tracesy[start1:end1, selected_animal[0]]
    
    # replace currental animal info
    tracesx[start1:end1, selected_animal[0]] = tempx[start1:end1, switch_id]
    tracesy[start1:end1, selected_animal[0]] = tempy[start1:end1, switch_id]
  
    # update plots
    update(current_index[0])
    
animal_switch.on_clicked(animal_switch_function)


# search for the next continuous chunk of red
def next_(event):
    
    # 
    for k in range(current_index[0], n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)
    
    for k in range(start_nan, n_frames, 1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            end_nan=k
            break

    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = end_nan
    sfreq.val = current_index[0]
    fig.canvas.draw_idle()

    print ("..... to : ", current_index[0])
    update(current_index[0])

    
    # search for the next continuous chunk of red

    
def forward_function(event):
    
    # 
    current_index[0]+=1
    sfreq.val = current_index[0]

    update(current_index[0])
    fig.canvas.draw_idle()
    
def backward_function(event):
    
    # 
    current_index[0]-=1
    sfreq.val = current_index[0]

    update(current_index[0])
    fig.canvas.draw_idle()

        
def del_jumps_function(event):
    
    global tracesx, tracesy
    # 
    print ("DELETING JUMPS FUCNTION")
    tracesx, tracesy = delete_jumps(tracesx,tracesy)

    update(current_index[0])
    fig.canvas.draw_idle()
    
    
def previous(event):
    
    # 
    start_nan=0
    for k in range(current_index[0], 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_nan = k
            break
    print ("searching... start_nan: ", start_nan)

    end_nan=0
    for k in range(start_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]])==False:
            end_nan=k
            break

    start_data =0
    for k in range(end_nan, 0, -1):
        if np.isnan(tracesx[k,selected_animal[0]]):
            start_data=k
            break
            
    print ("Fast forwarding from: ", current_index[0])
    current_index[0] = start_data+1
    sfreq.val = current_index[0]

    print ("..... to : ", current_index[0])
    update(current_index[0])
    fig.canvas.draw_idle()

    
def save(event):
    
    np.savez('/home/cat/traces_fixed', 
             tracesx = tracesx,
             tracesy = tracesy)

        
def load_function(event):
    
    global tracesx, tracesy
    
    data = np.load('/home/cat/traces_fixed.npz')
    tracesx = data['tracesx']
    tracesy = data['tracesy']
    
    print ("loaded traces")

def interpolate_function(event):
    global tracesx, tracesy
    
    dist_threshold = 100.
    time_threshold = 10
    tracesx, tracesy = interpolate_traces(tracesx, tracesy, 
                                          dist_threshold,
                                         time_threshold)
    
    print ("INTERPOLATING TRACES, THRESHOLD: ", threshold)
    
def export_movie(movie):
    clrs = ['blue','red','cyan','green']


    size_vid = np.array([1280,1024])
    dot_size = 16
    
    out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
    fname_out = (out_dir+'video_labeled_exported_fixed.avi')
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)

    # initialize start and ends of video to be analyzed
    start = 0
    #end = start + 10*25
    #end = tracesx.shape[1]
    end = 3000
    original_vid = cv2.VideoCapture(video_name)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    for k in range(start,end,1):    
        if k %100==0:
            print (k)

        ret, frame = original_vid.read()
        if ret==False:
            break
#         if scaling:
#             frame_cropped = frame.copy()[::4,::4]
#         else:
#             frame_cropped = frame.copy()

        # Loop over animals
        for p in range(0, tracesx.shape[1], 1):
            x = tracesx[k,p]
            y = tracesy[k,p]
            # plot large bump
            if np.isnan(x):
                continue
            x = int(x)
            y = int(y)
            
            frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(clrs[p]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()
    cv2.destroyAllWindows()


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axnext = plt.axes([0.01, 0.55, 0.04, 0.04])
bnext = Button(axnext, 'Next\nSegment')
bnext.on_clicked(next_)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axprev = plt.axes([0.01, 0.5, 0.04, 0.04])
bprev = Button(axprev, 'Previous\nSegment')
bprev.on_clicked(previous)
#bprev = Button(axprev, 'Previous')
#bprev.on_clicked(callback.prev)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axsave = plt.axes([0.01, 0.4, 0.04, 0.04])
bsave = Button(axsave, 'Save\nTraces')
bsave.on_clicked(save)

#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axload = plt.axes([0.01, 0.35, 0.04, 0.04])
bload = Button(axload, 'Load\nTraces')
bload.on_clicked(load_function)


#axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
axforward = plt.axes([0.01, 0.3, 0.04, 0.04])
bforward = Button(axforward, 'Step\nForward')
bforward.on_clicked(forward_function)

# BACK BUTTON
axback = plt.axes([0.01, 0.25, 0.04, 0.04])
bback = Button(axback, 'Step\nBackward')
bback.on_clicked(backward_function)
#bprev = Button(axprev, 'Previous')

# DELETE JUMPS BUTTON
axdel_jumps = plt.axes([0.01, 0.2, 0.04, 0.04])
bdeljumps = Button(axdel_jumps, 'Fix\nJumps')
bdeljumps.on_clicked(del_jumps_function)
#bprev = Button(axprev, 'Previous')


# Export movie
axmovie = plt.axes([0.01, 0.1, 0.04, 0.04])
bmovie = Button(axmovie, 'Export\nMovie')
bmovie.on_clicked(export_movie)
#bprev = Button(axprev, 'Previous')


# INTERPOLATE
axinterpolate = plt.axes([0.01, 0.05, 0.04, 0.04])
binterpoloate = Button(axinterpolate, 'Interpolate\nTraces')
binterpoloate.on_clicked(interpolate_function)
#bprev = Button(axprev, 'Previous')

plt.show()

current indx updated: [5085]
current indx updated: [5095]
current indx updated: [5190]
current indx updated: [5209]
current indx updated: [5114]
current indx updated: [5076]
current indx updated: [5037]
current indx updated: [4980]
current indx updated: [4952]
current indx updated: [4895]
current indx updated: [4866]
current indx updated: [4838]
current indx updated: [4828]
current indx updated: [4819]
current indx updated: [5076]
current indx updated: [5114]
current indx updated: [5133]
current indx updated: [5190]
current indx updated: [5218]
current indx updated: [5256]
(16572, 4)
INTERPOLATING TRACES, THRESHOLD:  10.0
current indx updated: [5247]
current indx updated: [5218]
current indx updated: [5209]
current indx updated: [5199]
current indx updated: [5190]


In [36]:
data1 = np.load('/home/cat/traces_fixed.npy.npz')
print (data1['tracesx'])

[[ 324.056        nan       nan       nan]
 [ 324.253        nan       nan       nan]
 [ 324.365        nan       nan       nan]
 ...
 [ 221.123   859.061  1179.818   275.199 ]
 [ 234.9225  859.061  1179.71    280.644 ]
 [ 219.568   859.061  1179.6125  278.984 ]]
